Notebook to test random stuff

In [1]:
import pandas as pd 
import numpy as np
import torch
from torch.utils.data import Dataset
from torch import nn

from library.util import bootstrap_labeled_sets
from library.datasets import Binned_Signal_Dataset

In [2]:
dset = Binned_Signal_Dataset(
    name="test_test", 
    level="gen", 
    split="train", 
    save_dir="../../state/new_physics/data/processed",
)

regenerate = False
if regenerate:
    dset.generate(
        raw_trials=range(1,2), 
        raw_signal_dir="../../state/new_physics/data/raw/signal",
        q_squared_veto=True,
        std_scale=True,
        balanced_classes=True,
        shuffle=True,
    )

dset.load()

Loaded features of shape: torch.Size([333212, 4]).
Loaded labels of shape: torch.Size([333212]).
Loaded bin values of shape: torch.Size([44]).


In [4]:
bootstrap_labeled_sets(dset.features, dset.labels, 4, 3, reduce_labels=False)

(tensor([[[-1.2115e+00,  1.0814e+00,  2.7655e-01,  1.4463e+00],
          [ 1.0363e+00,  1.5488e+00, -3.1950e-01, -1.1397e+00],
          [ 1.2986e+00,  1.3864e+00, -1.2947e+00, -1.2184e+00],
          [ 2.6042e-01, -7.6073e-01, -8.7166e-01, -8.4121e-01]],
 
         [[ 7.5175e-01, -3.8344e-01, -8.4719e-01,  1.0520e+00],
          [-5.9420e-01, -5.4488e-01, -4.4963e-01, -1.5366e+00],
          [-1.6558e+00,  1.1293e+00,  1.1558e+00,  1.6752e+00],
          [-5.5138e-01,  1.1689e+00,  8.1983e-01, -1.5382e+00]],
 
         [[-1.7240e+00,  1.0200e+00,  1.1841e+00, -1.7235e+00],
          [ 1.5617e+00,  4.5351e-01,  1.0105e+00,  1.4910e+00],
          [ 1.4340e+00,  2.5806e-01, -1.4029e+00,  1.5940e+00],
          [-1.4707e+00, -1.2183e+00, -1.1384e+00,  1.1252e+00]],
 
         ...,
 
         [[ 9.0420e-01, -6.8042e-01, -1.3912e+00,  1.6463e+00],
          [ 1.3927e+00, -2.3203e-01, -6.9661e-01, -1.5294e+00],
          [-2.0110e-01, -7.0858e-01, -1.2647e+00,  6.7458e-01],
          [-1.2

In [2]:
a = torch.tensor([0,1,2])

In [3]:
a

tensor([0, 1, 2])

In [7]:
a.unsqueeze(0).unsqueeze(0).shape

torch.Size([1, 1, 3])

In [8]:
from library.datasets import make_image

In [26]:
feat = torch.tensor([[1,-0.5,0.5,2], [2,-0.5,0.5,1], [2,1,1,6]])
feat

tensor([[ 1.0000, -0.5000,  0.5000,  2.0000],
        [ 2.0000, -0.5000,  0.5000,  1.0000],
        [ 2.0000,  1.0000,  1.0000,  6.0000]])

In [28]:
torch.nan_to_num(make_image(feat.numpy(), n_bins=2))

tensor([[[0.0000, 0.0000],
         [1.5000, 0.0000]],

        [[0.0000, 0.0000],
         [0.0000, 2.0000]]], dtype=torch.float64)

In [2]:
a = torch.tensor([1,2,1,2,3,3])

In [7]:
a == a[0]

tensor([True, True, True, True, True])

In [3]:
from library.util import get_num_per_unique_label

In [4]:
get_num_per_unique_label(a)

2

In [9]:
d = {80_000 : "a", 20 : "b"}

In [15]:
d = {i : 2*i for i in [2,4,6]}
d


{2: 4, 4: 8, 6: 12}